In [1]:
import hopsworks
from hsfs.feature import Feature
import snowflake.connector

proj = hopsworks.login(project="kimberly")
fs = proj.get_feature_store()

/home/jdowling/anaconda3/envs/book/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/644244
Connected. Call `.close()` to terminate connection gracefully.


In [2]:
github_most_starts_fg = fs.get_external_feature_group(
    name="github_most_stars",
    version=1
)

github_most_starts_fg.read()

/arrow/cpp/src/arrow/status.cc:137: DoAction result was not fully consumed: Cancelled: Flight cancelled call, with message: CANCELLED. Detail: Cancelled


Finished: Reading data from Hopsworks, using ArrowFlight (3.10s) 


,repo_name,sum_stars
0,significant-gravitas/autogpt,156550
1,automatic1111/stable-diffusion-webui,107157
2,f/awesome-chatgpt-prompts,100513
3,base-org/chains,87616
4,practical-tutorials/project-based-learning,87320
...,...,...
95,metatimeofficial/chainlist,27255
96,mlabonne/llm-course,27133
97,lllyasviel/controlnet,26914
98,microsoft/ai-for-beginners,26878


In [3]:
bike_stations = fs.get_feature_group(
    name="chicago_bike_stations",
    version=1
)

bike_station_status = fs.get_feature_group(
    name="chicago_bike_station_status",
    version=1
)

In [17]:
# select the features for your model
selected_features = bike_station_status.select(['station_status','num_bikes_available']).join(bike_stations.select(['station_type', 'capacity', 'has_kiosk']), left_on="id", right_on="station_id")
selected_features.show(10)

Finished: Reading data from Hopsworks, using ArrowFlight (5.34s) 


,station_status,num_bikes_available,station_type,capacity,has_kiosk
0,"""active""",3,"""lightweight""",9,false
1,"""active""",1,"""lightweight""",9,false
2,"""active""",2,"""lightweight""",9,false
3,"""active""",0,"""lightweight""",9,false
4,"""active""",4,"""lightweight""",9,false
5,"""active""",0,"""lightweight""",9,false
6,"""active""",0,"""lightweight""",9,false
7,"""active""",0,"""lightweight""",9,false
8,"""active""",0,"""lightweight""",12,false
9,"""active""",1,"""lightweight""",9,false


In [18]:
fv = fs.get_or_create_feature_view(name="bikes", 
                                   version=1,
                                   description="Predict bike availability",
                                   query=selected_features,
                                   labels=["num_bikes_available"]
                                  )

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/644244/fs/640067/fv/bikes/version/1


In [21]:
X_train, X_test, y_train, y_test = fv.train_test_split(test_size=0.1)

Finished: Reading data from Hopsworks, using ArrowFlight (3.75s) 


In [22]:
X_train

,station_status,station_type,capacity,has_kiosk
0,"""active""","""lightweight""",9,false
1,"""active""","""lightweight""",9,false
2,"""active""","""lightweight""",9,false
3,"""active""","""lightweight""",9,false
6,"""active""","""lightweight""",9,false
...,...,...,...,...
7193,"""active""","""lightweight""",8,false
7194,"""active""","""lightweight""",9,false
7195,"""active""","""lightweight""",6,false
7196,"""active""","""lightweight""",9,false


In [ ]:
# Train your model ....